In [90]:
import scipy.io
import pandas as pd
import numpy as np
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt



import classification
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
import numpy as np
import os
import sys
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import metrics
import itertools
import classification
#import other python scripts for further anlaysis
#import reshape
#import results
import warnings
warnings.filterwarnings("ignore")
# Initialization of directory information:
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
#using less conservative fc matrices
dataDir = thisDir + 'data/mvpa_data/'
framesDir = thisDir + 'data/mvpa_data/tmask_frames/'
outDir = thisDir + 'output/mLmin/results/ridge/frames/'

#dataDir = thisDir + 'data/mvpa_data/'
#outDir = thisDir + 'output/mL/'
# Subjects and tasks
taskList=['mixed', 'motor','mem']
#taskList=['glass','semantic', 'motor','mem']
#subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
subList=['MSC02','MSC04','MSC05','MSC06','MSC07']
#all possible combinations of subs and tasks
subsComb=(list(itertools.permutations(subList, 2)))
tasksComb=(list(itertools.permutations(taskList, 2)))
#DS combination
DSvars=list(itertools.product(list(subsComb),list(taskList)))
##SS combination
SSvars=list(itertools.product(list(subList),list(tasksComb)))
#BS combination
BSvars=list(itertools.product(list(subsComb),list(tasksComb)))

#only training on memory 
#frames
#frameList=[5,10,15,20,25,30,40,50,60,70,80,90,100,125,
frameList=[150,175,200,225,250,275,300,325,350]

def calc_frames():
    CV_acc=[]
    SS_acc=[]
    DS_acc=[]
    BS_acc=[]
    for f in frameList:
        #classifyCV(f)
        #classifySS(f)
        #classifyDS(f)
        classifyBS(f)
        print ('Now finishing with '+str(f))
        """
        CV_acc.append(classifyCV(f))
        SS_acc.append(classifySS(f))
        DS_acc.append(classifyDS(f))
        BS_acc.append(classifyBS(f))
    
    tmp_CV=pd.DataFrame({'Frames':frameList, 'Acc':CV_acc,'Analysis':'CV'})
    tmp_SS=pd.DataFrame({'Frames':frameList, 'Acc':SS_acc,'Analysis':'SS'})
    tmp_DS=pd.DataFrame({'Frames':frameList, 'Acc':DS_acc,'Analysis':'DS'})
    tmp_BS=pd.DataFrame({'Frames':frameList, 'Acc':BS_acc,'Analysis':'BS'})
    df_all=[tmp_CV,tmp_SS,tmp_DS,tmp_BS]
    allFrames=pd.concat(df_all)
    allFrames.to_csv(outDir+'allFrames_rest.csv')
    
    """
def classifyDS(frames):
    """
    Classifying different subjects (DS) along the same task

    Parameters
    -------------
    classifier : str
            The statistical method used for classification
    analysis : str
            The type of analysis to be conducted

    Returns
    -------------
    dfDS : DataFrame
        Dataframe consisting of average accuracy across all subjects

    """
    acc_scores_per_task=[]
    train=[]
    test=[]
    tmp_df=pd.DataFrame(DSvars, columns=['sub','task'])
    dfDS=pd.DataFrame()
    #dfDS[['train_sub','test_sub']]=pd.DataFrame(tmp_df['sub'].tolist())
    #dfDS['task']=tmp_df['task']
    for train_sub in subList:
        for test_sub in subList:
            if train_sub==test_sub:
                continue
            score=model(frames,'DS', train_sub=train_sub, test_sub=test_sub, train_task='mem', test_task='mem')
            acc_scores_per_task.append(score)
            train.append(train_sub)
            test.append(test_sub)
    dfDS['acc']=acc_scores_per_task
    dfDS['train']=train
    dfDS['test']=test
    dfDS['frames']=frames
    dfDS['analysis']='DS'
    dfDS.to_csv(outDir+'DS/'+str(frames)+'DS.csv',index=False)
    #DS_acc=statsACC(dfDS, 'DS')
    #return DS_acc
    
def classifySS(frames):
    """
    Classifying the same subject (SS) along a different task

    Parameters
    -------------
    classifier : str
            The statistical method used for classification
    analysis : str
            The type of analysis to be conducted

    Returns
    -------------
    dfSS : DataFrame
        Dataframe consisting of average accuracy across all subjects

    """
    acc_scores_per_task=[]
    train=[]
    tmp_df=pd.DataFrame(SSvars, columns=['sub','task'])
    dfSS=pd.DataFrame()
    #dfSS[['train_task','test_task']]=pd.DataFrame(tmp_df['task'].tolist())
    #dfSS['sub']=tmp_df['sub']
    for sub in subList:
        score=model(frames,'SS', train_sub=sub, test_sub=sub, train_task='mem', test_task='mixed')
        acc_scores_per_task.append(score)
        train.append(sub)
    dfSS['acc']=acc_scores_per_task
    dfSS['train']=train
    dfSS['frames']=frames
    dfSS['analysis']='SS'
    dfSS.to_csv(outDir+'SS/'+str(frames)+'SS.csv',index=False)
    #SS_acc=statsACC(dfSS, 'SS')
    #return SS_acc
def classifyBS(frames):
    """
    Classifying different subjects (BS) along different tasks

    Parameters
    -------------
    classifier : str
            The statistical method used for classification
    analysis : str
            The type of analysis to be conducted

    Returns
    -------------
    dfBS : DataFrame
        Dataframe consisting of average accuracy across all subjects

    """
    acc_scores_per_task=[]
    train=[]
    test=[]
    #tmp_df=pd.DataFrame(BSvars, columns=['sub','task'])
    dfBS=pd.DataFrame()
    #dfBS[['train_task','test_task']]=pd.DataFrame(tmp_df['task'].tolist())
    #dfBS[['train_sub', 'test_sub']]=pd.DataFrame(tmp_df['sub'].tolist())
    for train_sub in subList:
        for test_sub in subList:
            if train_sub==test_sub:
                continue
        score=model(frames,'BS', train_sub=train_sub, test_sub=test_sub, train_task='mem', test_task='mixed')
        acc_scores_per_task.append(score)
        train.append(train_sub)
        test.append(test_sub)
    dfBS['acc']=acc_scores_per_task
    dfBS['train']=train
    dfBS['test']=test
    dfBS['frames']=frames
    dfBS['analysis']='BS'
    dfBS.to_csv(outDir+'BS/'+str(frames)+'BS.csv',index=False)
    #BS_acc=statsACC(dfBS, 'BS')
    #return BS_acc


def classifyCV(frames):
    """
    Classifying same subjects (CV) along the same task

    Parameters
    -------------
    classifier : str
            The statistical method used for classification
    analysis : str
            The type of analysis to be conducted

    Returns
    -------------
    dfCV : DataFrame
        Dataframe consisting of average accuracy across all subjects

        """

    clf=RidgeClassifier()
    cvTable=[]
    acc_scores=[]
    for sub in subList:
        taskFC=matFiles(framesDir+'mem/'+str(frames)+'/'+sub+'_parcel_corrmat.mat')
        restFC=matFiles(framesDir+'rest/'+str(frames)+'/'+sub+'_parcel_corrmat.mat')
        if taskFC.size==1 or restFC.size==1:
            mu='9999'
            print('subject '+sub+ ' has no data for frame '+str(frames))
        else:
            folds=taskFC.shape[0]
            print('subject '+sub)
            x_train, y_train=classification.concateFC(taskFC, restFC)
            CVscores=cross_val_score(clf, x_train, y_train, cv=folds)
            mu=CVscores.mean()
        acc_scores.append(mu)
    dfCV=pd.DataFrame({'train':subList, 'acc':acc_scores, 'frames':frames, 'analysis':'CV'})
    dfCV.to_csv(outDir+'CV/'+str(frames)+'CV.csv',index=False)
    #dfCV=pd.DataFrame({'sub':subList, 'acc':acc_scores}).set_index('sub')
    #CV_acc=statsACC(dfCV, 'CV')
    #return CV_acc
    
"""
def statsACC(df, analysis):
    if analysis=='CV':
        print('cross validation stats')
        mu=df['acc'].mean()
    elif analysis=='SS':
        print('same sub stats')
        mu=df['acc'].mean()
    elif analysis=='DS':
        print('diff sub stats')
        mu=df['acc'].mean()
    elif analysis=='BS':
        print('diff sub diff task stats')
        mu=df['acc'].mean()
    else:
        print('skipping stats')
    return mu
"""
def matFiles(df='path'):
    """
    Convert matlab files into upper triangle np.arrays
    Parameters
    -----------
    df : str
        Path to file
    Returns
    -----------
    ds : 2D upper triangle FC measures in (roi, days) format

    """
    #Consistent parameters to use for editing datasets
    nrois=333
    #Load FC file
    fileFC=scipy.io.loadmat(df)

    
    #Convert to numpy array
    fileFC=np.array(fileFC['parcel_corrmat'])
    #Replace nans and infs with zero
    fileFC=np.nan_to_num(fileFC)
    #if you dont have enough sessions return an empty np array 
    if fileFC.ndim<3:
        return np.empty([])
    else:
        nsess=fileFC.shape[2]
    #Index upper triangle of matrix
        mask=np.triu_indices(nrois,1)
        ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
        count=0
    #Loop through all 10 days to reshape correlations into linear form
        for sess in range(nsess):
            tmp=fileFC[:,:,sess]
            ds[count]=tmp[mask]
            count=count+1
        return ds



def model(frames,analysis, train_sub, test_sub, train_task, test_task):
    """
    Preparing machine learning model with appropriate data

    Parameters
    -------------
    classifier : str
            The statistical method used for classification
    analysis : string
            The type of analysis to be conducted
    train_sub : str
            Subject name for training
    test_sub : str
            Subject name for testing
    train_task : str
            Task name for training
    test_task : str
            Task name for testing

    Returns
    -------------
    total_score : float
            Average accuracy of all folds

    """

    clf=RidgeClassifier()
    df=pd.DataFrame()
    taskFC=matFiles(framesDir+'mem/'+str(frames)+'/'+train_sub+'_parcel_corrmat.mat')
    restFC=matFiles(framesDir+'rest/'+str(frames)+'/'+train_sub+'_parcel_corrmat.mat')
    if taskFC.size==1 or restFC.size==1:
        total_score='9999'
        print('subject '+train_sub+ ' has no data for frame '+str(frames))
    #if your subs are the same
    elif train_sub==test_sub:
        test_taskFC=classification.matFiles(dataDir+test_task+'/'+test_sub+'_parcel_corrmat.mat')
        total_score=classification.CV_folds(clf, analysis, taskFC, restFC, test_taskFC, restFC)
    else:
        test_taskFC=classification.matFiles(dataDir+test_task+'/'+test_sub+'_parcel_corrmat.mat')
        test_restFC=classification.matFiles(dataDir+'rest/'+test_sub+'_parcel_corrmat.mat')
        total_score=classification.CV_folds(clf, analysis, taskFC, restFC, test_taskFC, test_restFC)
    return total_score

In [91]:
calc_frames()

Now finishing with 150
Now finishing with 175
Now finishing with 200
Now finishing with 225
Now finishing with 250
Now finishing with 275
Now finishing with 300
Now finishing with 325
Now finishing with 350
